In [94]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from shapely.affinity import scale
from shapely.geometry import Point
import pandas as pd
from pygbif.species import name_backbone
import geopandas as gpd


file_dir=('/Users/maddie/Projects/CPSC_597/data')

In [95]:
#Create the dataframes to be concatenated and filtered
occ_all_species = pd.read_csv(file_dir+"/data_raw/gbif_data_raw/occurrences_all_species.csv", low_memory= False)
df = occ_all_species[occ_all_species['label'].str.contains(" ")]

#Get unique label names
unique_labels=df["label"].unique()

names = []
back_key =[]
remaining_labels=[]

#Get backbone associated species names and taxon keys
for item in unique_labels:
    if "species" in name_backbone(item):
        i = name_backbone(item)['species']
        j = name_backbone(item)['speciesKey']
        names.append(i)
        back_key.append(j)
    else:
        remaining_labels.append(item)
        
for item in remaining_labels:
    value=name_backbone(item)['taxonKey']
    back_key.append(value)
    names.append(item)
    
#Put into DataFrame
df=pd.DataFrame({"label": unique_labels,"back_key": back_key,"species": names},columns=["label","back_key","species"])

In [96]:
#Concatenate with occurrence data, dataframe, drop na's 
df2=pd.merge(occ_all_species,df,how="left",on="label")

df2 = df2[pd.notnull(df2['species_x'])]
df2 = df2[pd.notnull(df2['decimalLatitude'])]
df2 = df2[pd.notnull(df2['decimalLongitude'])]

print("df2 without na's n.rows:", len(df2.index))

df2["back_key"]=df2["back_key"].astype(int)

df2 without na's n.rows: 250264


In [97]:
#list of species
species = df2["species_x"].unique()
species.sort()

#save separate dataframe for each species as csv file 
for spec in species:
    data=df2.loc[df2['species_x'] == spec]
    if len(data.index)>= 10:
        spec=spec.replace(" ","_")
        print("%s"%spec, len(data.index))
        data.to_csv(file_dir+'/data_raw/gbif_data_raw/%s_gbif_raw.csv'%spec)


Citharichthys_sordidus 31634
Engraulis_mordax 24710
Paralichthys_californicus 5340
Scomber_japonicus 20855
Thunnus_alalunga 35567
Xiphias_gladius 132158
